In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('cereal.csv', sep=';')
df.drop(df.index[0], inplace = True)


In [5]:
X = df[['name', 'mfr', 'type', 'calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'shelf', 'weight', 'cups']]
y = df['rating']

In [6]:
X.drop(["name","mfr"],axis=1,inplace=True)
X = pd.get_dummies(X, columns=['type'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
X

,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,type_C,type_H
1,70,4,1,130,10,5,6,280,25,3,1,0.33,1,0
2,120,3,5,15,2,8,8,135,0,3,1,1,1,0
3,70,4,1,260,9,7,5,320,25,3,1,0.33,1,0
4,50,4,0,140,14,8,0,330,25,3,1,0.5,1,0
5,110,2,2,200,1,14,8,-1,25,3,1,0.75,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,110,2,1,250,0,21,3,60,25,3,1,0.75,1,0
74,110,1,1,140,0,13,12,25,25,2,1,1,1,0
75,100,3,1,230,3,17,3,115,25,1,1,0.67,1,0
76,100,3,1,200,3,17,3,110,25,1,1,1,1,0


In [8]:
X1=X.drop(["sodium","sugars"],axis=1)

In [9]:
X1

,calories,protein,fat,fiber,carbo,potass,vitamins,shelf,weight,cups,type_C,type_H
1,70,4,1,10,5,280,25,3,1,0.33,1,0
2,120,3,5,2,8,135,0,3,1,1,1,0
3,70,4,1,9,7,320,25,3,1,0.33,1,0
4,50,4,0,14,8,330,25,3,1,0.5,1,0
5,110,2,2,1,14,-1,25,3,1,0.75,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
73,110,2,1,0,21,60,25,3,1,0.75,1,0
74,110,1,1,0,13,25,25,2,1,1,1,0
75,100,3,1,3,17,115,25,1,1,0.67,1,0
76,100,3,1,3,17,110,25,1,1,1,1,0


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 15)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X1, y, test_size = 0.2, random_state = 15)

In [11]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

model2 = LinearRegression()
model2.fit(X_train2, y_train2)

LinearRegression()

In [12]:
y_pred = model.predict(X_test)
y_pred2 = model2.predict(X_test2)

In [13]:
def estimate_coef_multiple_linear_regression(X_train,y_train):
    X_train = X_train.apply(pd.to_numeric, errors='coerce')
    y_train = y_train.apply(pd.to_numeric, errors='coerce')
    
    return np.linalg.inv(X_train.transpose().dot(X_train)).dot(X_train.transpose()).dot(y_train)

In [14]:
def estimate_coef_multiple_linear_regression2(X_train2,y_train2):
    X_train2 = X_train2.apply(pd.to_numeric, errors='coerce')
    y_train2 = y_train2.apply(pd.to_numeric, errors='coerce')
    
    return np.linalg.inv(X_train2.transpose().dot(X_train2)).dot(X_train2.transpose()).dot(y_train2)

In [15]:
coef = estimate_coef_multiple_linear_regression(X_train,y_train)
X_test = X_test.apply(pd.to_numeric, errors='coerce')
y_test = y_test.apply(pd.to_numeric, errors='coerce')
predictions = np.dot(X_test,coef)

In [16]:
coef2 = estimate_coef_multiple_linear_regression(X_train2,y_train2)
X_test2 = X_test2.apply(pd.to_numeric, errors='coerce')
y_test2 = y_test2.apply(pd.to_numeric, errors='coerce')
predictions2 = np.dot(X_test2,coef2)

In [17]:
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

0.9999999999999997

In [18]:
r2_score(y_test2, predictions2)

0.8767937741693388

###Found out that calories, fiber, and sugars has high R2 values. 

In [19]:
from numpy import log as ln
resid = y_test.astype(float) - y_pred
sse = sum(resid**2)
k1=14
N1=len(y_test)
AIC= N1 * ln(sse/N1) + 2*k1
print("AIC of Model1:",AIC)
##for model2
resid2 = y_test2.astype(float) - y_pred2
sse2 = sum(resid2**2)
k2=12
N2=len(y_test2)
AIC2= N2 * ln(sse2/N2) + 2*k2
print("AIC of Model2:",AIC2)

AIC of Model1: -450.4096783460508
AIC of Model2: 81.8454551050797


In [20]:
BIC1 = N1 * ln(sse/N1) + k1*ln(N1)

#BIC for model with 4 predictors
BIC2 = N2 * ln(sse2/N2) + k2*ln(N2)

print("BIC of Model1:", BIC1)
print("BIC of Model2:", BIC2)

BIC of Model1: -439.59343623469385
BIC of Model2: 91.11651977195706


#Report Summary

Model  | Metrices | Score
-------------------|------------------|-----|
Model 1      | Adj R2 | 99.99%
Model 2      | Adj R2 | 87.67%
Model 1     | AIC | -450
Model 2       | AIC | 81.84
Model 1       | BIC | -439
Model 2       | BIC | 91.11


###Model with highest Adjusted R-squared values and least AIC and BIC value is considered as best Model. 

**Hence Model 1(full model) is Better than Model 2(model
with two less variables)**